# Minimum Variance Portfolio Optimization

This notebook demonstrates how to construct a minimum variance portfolio using the Portfolio Optimization Testbed.

The minimum variance portfolio is the portfolio with the lowest possible risk (volatility) without regard to expected returns. It is located at the leftmost point of the efficient frontier.

## 1. Import Required Modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from portopt.core.problem import PortfolioOptProblem
from portopt.core.objective import MinimumVarianceObjective
from portopt.constraints.basic import FullInvestmentConstraint, LongOnlyConstraint
from portopt.solvers.classical import ClassicalSolver
from portopt.utils.data import TestDataGenerator
from portopt.metrics.risk import calculate_portfolio_volatility
from portopt.utils.visualization import plot_efficient_frontier, plot_weights

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["font.size"] = 12

## 2. Generate Test Data

We'll use the `TestDataGenerator` to create synthetic data for our example. Alternatively, you could load your own data from a CSV file or other source.

In [ ]:
# Create a data generator with a fixed seed for reproducibility
generator = TestDataGenerator(seed=42)

# Generate a realistic problem with 10 assets and 252 periods (1 year of daily returns)
problem = generator.generate_realistic_problem(
    n_assets=10,
    n_periods=252,
    n_factors=3,
    n_industries=4
)

# Extract the returns data
returns = problem.returns

# Calculate mean returns and convert to annualized returns (assuming daily data)
mean_returns = np.mean(returns, axis=0) * 252

# Calculate the covariance matrix and annualize it
cov_matrix = np.cov(returns, rowvar=False) * 252

# Display basic statistics
print(f"Number of assets: {problem.n_assets}")
print(f"Number of time periods: {problem.n_periods}")
print(f"\nAnnualized mean returns:")
for i, ret in enumerate(mean_returns):
    print(f"Asset {i+1}: {ret:.4f}")

print(f"\nAnnualized volatilities:")
for i, vol in enumerate(np.sqrt(np.diag(cov_matrix))):
    print(f"Asset {i+1}: {vol:.4f}")

## 3. Visualize the Returns Data

Let's visualize the returns data to get a better understanding of our assets.

In [ ]:
# Create a DataFrame for easier plotting
returns_df = pd.DataFrame(returns, columns=[f"Asset {i+1}" for i in range(problem.n_assets)])

# Plot the returns distribution
plt.figure(figsize=(14, 7))
sns.boxplot(data=returns_df)
plt.title("Distribution of Daily Returns")
plt.ylabel("Return")
plt.grid(True, alpha=0.3)
plt.show()

# Plot the correlation matrix
plt.figure(figsize=(10, 8))
correlation_matrix = np.corrcoef(returns, rowvar=False)
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1,
            xticklabels=[f"Asset {i+1}" for i in range(problem.n_assets)],
            yticklabels=[f"Asset {i+1}" for i in range(problem.n_assets)])
plt.title("Correlation Matrix of Asset Returns")
plt.tight_layout()
plt.show()

## 4. Define the Minimum Variance Portfolio Optimization Problem

Now, let's set up the optimization problem to find the minimum variance portfolio.

In [ ]:
# Define constraints
constraints = [
    FullInvestmentConstraint(),  # Sum of weights = 1
    LongOnlyConstraint()         # No short selling (weights >= 0)
]

# Define the objective function
objective = MinimumVarianceObjective()

# Create a solver
solver = ClassicalSolver(max_iterations=100, tolerance=1e-8)

# Solve the problem
result = solver.solve(problem, constraints=constraints, objective=objective)

# Extract the optimal weights
min_var_weights = result.weights

# Calculate portfolio statistics
min_var_return = np.dot(mean_returns, min_var_weights)
min_var_volatility = calculate_portfolio_volatility(min_var_weights, cov_matrix)

print("Minimum Variance Portfolio:")
print(f"Expected Annual Return: {min_var_return:.4f}")
print(f"Expected Annual Volatility: {min_var_volatility:.4f}")
print(f"Sharpe Ratio (assuming risk-free rate = 0): {min_var_return / min_var_volatility:.4f}")

print("\nOptimal Weights:")
for i, weight in enumerate(min_var_weights):
    if weight > 0.001:  # Only show significant weights
        print(f"Asset {i+1}: {weight:.4f}")

## 5. Visualize the Minimum Variance Portfolio Weights

Let's visualize the weights of our minimum variance portfolio.

In [ ]:
# Plot the weights
plt.figure(figsize=(12, 6))
plt.bar(range(len(min_var_weights)), min_var_weights)
plt.xlabel("Asset")
plt.ylabel("Weight")
plt.title("Minimum Variance Portfolio Weights")
plt.xticks(range(len(min_var_weights)), [f"Asset {i+1}" for i in range(len(min_var_weights))])
plt.grid(True, alpha=0.3)
plt.show()

# Create a pie chart for weights > 1%
significant_weights = [w if w > 0.01 else 0 for w in min_var_weights]
if sum(significant_weights) < 1.0:
    significant_weights.append(1.0 - sum(significant_weights))  # Add "Others" category
    labels = [f"Asset {i+1}" for i in range(len(min_var_weights)) if min_var_weights[i] > 0.01]
    labels.append("Others")
else:
    labels = [f"Asset {i+1}" for i in range(len(min_var_weights)) if min_var_weights[i] > 0.01]

plt.figure(figsize=(10, 8))
plt.pie(significant_weights, labels=labels, autopct='%1.1f%%', startangle=90, shadow=True)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.title("Minimum Variance Portfolio Allocation")
plt.show()

## 6. Compare with Equal Weight Portfolio

Let's compare our minimum variance portfolio with a simple equal weight portfolio.

In [ ]:
# Create equal weight portfolio
equal_weights = np.ones(problem.n_assets) / problem.n_assets

# Calculate portfolio statistics
equal_return = np.dot(mean_returns, equal_weights)
equal_volatility = calculate_portfolio_volatility(equal_weights, cov_matrix)

print("Equal Weight Portfolio:")
print(f"Expected Annual Return: {equal_return:.4f}")
print(f"Expected Annual Volatility: {equal_volatility:.4f}")
print(f"Sharpe Ratio (assuming risk-free rate = 0): {equal_return / equal_volatility:.4f}")

# Compare the portfolios
print("\nComparison:")
print(f"Volatility Reduction: {(equal_volatility - min_var_volatility) / equal_volatility * 100:.2f}%")
print(f"Return Difference: {(min_var_return - equal_return) / equal_return * 100:.2f}%")

# Plot comparison
plt.figure(figsize=(10, 6))
plt.scatter(equal_volatility, equal_return, marker='o', color='blue', s=200, label='Equal Weight')
plt.scatter(min_var_volatility, min_var_return, marker='o', color='red', s=200, label='Minimum Variance')
plt.xlabel('Expected Volatility')
plt.ylabel('Expected Return')
plt.title('Risk-Return Comparison')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

## 7. Generate the Efficient Frontier

Let's generate and plot the efficient frontier to see where our minimum variance portfolio lies.

In [ ]:
from portopt.core.objective import MeanVarianceObjective
from portopt.constraints.basic import TargetReturnConstraint

# Define a range of target returns
target_returns = np.linspace(min_var_return, max(mean_returns), 20)

# Initialize arrays to store the efficient frontier points
efficient_returns = []
efficient_volatilities = []

# For each target return, find the minimum variance portfolio
for target_return in target_returns:
    # Define constraints including the target return
    constraints = [
        FullInvestmentConstraint(),
        LongOnlyConstraint(),
        TargetReturnConstraint(target_return=target_return)
    ]
    
    # Solve the problem
    try:
        result = solver.solve(problem, constraints=constraints, objective=objective)
        weights = result.weights
        
        # Calculate portfolio return and volatility
        portfolio_return = np.dot(mean_returns, weights)
        portfolio_volatility = calculate_portfolio_volatility(weights, cov_matrix)
        
        efficient_returns.append(portfolio_return)
        efficient_volatilities.append(portfolio_volatility)
    except Exception as e:
        print(f"Optimization failed for target return {target_return:.4f}: {e}")

# Plot the efficient frontier
plt.figure(figsize=(12, 8))

# Plot individual assets
for i in range(problem.n_assets):
    plt.scatter(np.sqrt(cov_matrix[i, i]), mean_returns[i], marker='o', s=100, 
                label=f'Asset {i+1}')

# Plot the efficient frontier
plt.plot(efficient_volatilities, efficient_returns, 'b-', linewidth=3, label='Efficient Frontier')

# Plot the minimum variance portfolio
plt.scatter(min_var_volatility, min_var_return, marker='*', color='red', s=300, label='Minimum Variance')

# Plot the equal weight portfolio
plt.scatter(equal_volatility, equal_return, marker='s', color='green', s=200, label='Equal Weight')

plt.xlabel('Expected Volatility')
plt.ylabel('Expected Return')
plt.title('Efficient Frontier')
plt.grid(True, alpha=0.3)
plt.legend(loc='best')
plt.show()

## 8. Analyze Portfolio Diversification

Let's analyze how well our minimum variance portfolio is diversified.

In [ ]:
from portopt.metrics.risk import calculate_risk_contribution

# Calculate risk contribution for minimum variance portfolio
min_var_risk_contrib = calculate_risk_contribution(min_var_weights, cov_matrix)
min_var_risk_contrib_pct = min_var_risk_contrib / sum(min_var_risk_contrib)

# Calculate risk contribution for equal weight portfolio
equal_risk_contrib = calculate_risk_contribution(equal_weights, cov_matrix)
equal_risk_contrib_pct = equal_risk_contrib / sum(equal_risk_contrib)

# Plot risk contributions
plt.figure(figsize=(14, 7))

# Create a bar chart with two groups: minimum variance and equal weight
bar_width = 0.35
index = np.arange(problem.n_assets)

plt.bar(index, min_var_risk_contrib_pct, bar_width, label='Minimum Variance')
plt.bar(index + bar_width, equal_risk_contrib_pct, bar_width, label='Equal Weight')

plt.xlabel('Asset')
plt.ylabel('Risk Contribution (%)')
plt.title('Risk Contribution by Asset')
plt.xticks(index + bar_width / 2, [f'Asset {i+1}' for i in range(problem.n_assets)])
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate diversification metrics
min_var_concentration = np.sum(min_var_weights ** 2)  # Herfindahl-Hirschman Index
equal_concentration = np.sum(equal_weights ** 2)

min_var_risk_concentration = np.sum(min_var_risk_contrib_pct ** 2)
equal_risk_concentration = np.sum(equal_risk_contrib_pct ** 2)

print("Diversification Metrics:")
print(f"Minimum Variance Portfolio Weight Concentration: {min_var_concentration:.4f}")
print(f"Equal Weight Portfolio Weight Concentration: {equal_concentration:.4f}")
print(f"Minimum Variance Portfolio Risk Concentration: {min_var_risk_concentration:.4f}")
print(f"Equal Weight Portfolio Risk Concentration: {equal_risk_concentration:.4f}")

## 9. Backtest the Portfolio Performance

Let's simulate how our minimum variance portfolio would have performed over the historical period.

In [ ]:
# Calculate historical returns for both portfolios
min_var_historical_returns = np.dot(returns, min_var_weights)
equal_historical_returns = np.dot(returns, equal_weights)

# Calculate cumulative returns
min_var_cumulative = (1 + min_var_historical_returns).cumprod()
equal_cumulative = (1 + equal_historical_returns).cumprod()

# Plot cumulative returns
plt.figure(figsize=(14, 7))
plt.plot(min_var_cumulative, label='Minimum Variance Portfolio', linewidth=2)
plt.plot(equal_cumulative, label='Equal Weight Portfolio', linewidth=2)
plt.xlabel('Time Period')
plt.ylabel('Cumulative Return')
plt.title('Portfolio Performance Over Time')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

# Calculate performance metrics
from portopt.metrics.performance import calculate_sharpe_ratio, calculate_maximum_drawdown

min_var_sharpe = calculate_sharpe_ratio(min_var_historical_returns)
equal_sharpe = calculate_sharpe_ratio(equal_historical_returns)

min_var_drawdown = calculate_maximum_drawdown(min_var_cumulative)
equal_drawdown = calculate_maximum_drawdown(equal_cumulative)

print("Performance Metrics:")
print(f"Minimum Variance Portfolio Sharpe Ratio: {min_var_sharpe:.4f}")
print(f"Equal Weight Portfolio Sharpe Ratio: {equal_sharpe:.4f}")
print(f"Minimum Variance Portfolio Maximum Drawdown: {min_var_drawdown:.4f}")
print(f"Equal Weight Portfolio Maximum Drawdown: {equal_drawdown:.4f}")

## 10. Conclusion

In this notebook, we've demonstrated how to construct a minimum variance portfolio using the Portfolio Optimization Testbed. We've seen that the minimum variance portfolio typically has lower risk than an equal weight portfolio, though potentially at the cost of lower returns.

Key takeaways:
- The minimum variance portfolio is located at the leftmost point of the efficient frontier
- It provides the lowest possible portfolio volatility
- The weights are often concentrated in a few low-volatility assets
- Risk contributions may still be uneven
- Historical performance shows lower drawdowns compared to equal weighting

For more advanced portfolio optimization techniques, see the other examples in the Portfolio Optimization Testbed documentation.